# second assisgnment is: take a multiple pdf with text,image,table
1. ~~fetch the data from pdf~~
2. ~~at lesat there should be 200 pages~~
3. ~~if chunking(use the sementic chunking technique) required do chunking and then embedding~~
4. ~~store it inside the vector database(use any of them 1. mongodb 2. astradb 3. opensearch 4.milvus) ## i have not discuss then you need to explore~~
5. create a index with all three index machnism(Flat, HNSW, IVF) ## i have not discuss then you need to explore
6. ~~create a retriever pipeline~~
7. ~~check the retriever time(which one is fastet)~~
8. ~~print the accuray score of every similarity search~~
9. ~~perform the reranking either using BM25 or MMR ## i have not discuss then you need to explore~~
10. ~~then write a prompt template~~
11. ~~generte a oputput through llm~~
12. render that output over the DOCx ## i have not discuss then you need to explore
as a additional tip: you can follow rag playlist from my youtube

after completing it keep it on your github and share that link on my  mail id:
snshrivas3365@gmail.com

and share the assignment in your community chat as well by tagging krish and sunny

deadline is: till firday 9PM

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

True

In [2]:
def update_document_metadata(docs) :
    page = 0
    for doc in docs:
        doc.metadata = {
            'page_number': page
        }
        page += 1

In [3]:
# read from pdf file
loader = PyPDFLoader('data/combined.pdf')
documents = loader.load()
update_document_metadata(documents)

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = splitter.split_documents(documents)

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

In [12]:
from langchain_milvus import Milvus

URI = os.getenv('ZILLIZ_URL')

vector_store_flat = Milvus(
    embedding_function=embeddings,
    connection_args={'uri': URI, 'token': os.getenv('ZILLIZ_TOKEN'), 'db_name': 'milvus_flat_demo'},
    index_params={'index_type': 'FLAT', 'metric_type': 'COSINE'},
    consistency_level='Strong',
    vector_field=['dense'],
    drop_old=True
)

2025-06-05 22:22:06,748 [DEBUG][_create_connection]: Created new connection using: 1230856bd1874d44abec929a9a6d3d31 (async_milvus_client.py:599)


In [13]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(split_documents))]

In [14]:
vector_store_flat.add_documents(documents=split_documents, ids=uuids)

['8c9d3420-1849-4cc6-a091-5e53d900b22d',
 '0a6dd2e4-d78b-4108-bf6c-b690cb6b455d',
 'b35dbf97-a198-4c8e-ba9b-62ea410f36d7',
 '381dee28-b8cd-4b0d-b6b0-242592689356',
 '92b74740-0d6f-4ffc-952a-7967614bc8f4',
 '8c16a930-14e1-4526-9626-d89ecb42453d',
 'dcb3cb11-7f3f-4ea2-a63e-b494566ca5b9',
 '2ca51356-ec53-4c6e-9641-c727777fe95d',
 '8b2bb7f1-cd9c-4c69-a738-3f1dcc98a61e',
 '62434b38-83d5-4ca2-a565-90e19768eecb',
 'e095daec-3764-4fbe-a421-305858ae10fc',
 '9a1a4f72-738d-433e-a7b0-a3260c0376b7',
 'b4b7a9ea-9466-4f26-bbbc-1d746c0cdf8a',
 '5a313f24-3058-4999-a011-0dacc14e9db4',
 '8dc72e18-118d-44b6-9d8a-fecf6ce67c67',
 'e90d8907-cc4c-43ea-84ac-e88645afb6b8',
 '22c400f8-3ab0-4050-bb8c-d531f08bf762',
 'b4efef44-103c-4784-9abe-734c7f5defc9',
 '4d0ec383-cf88-480b-b91d-d0cb140d36ee',
 '0206eb44-6493-49c1-8da1-d5dc070d61e5',
 'e6154186-f129-4210-bea2-d6450a6f0fa6',
 '944117d1-d14b-44b8-871a-4e77e97883eb',
 '0677598e-9109-4561-8ad7-2ff83fc85b76',
 '213fa489-1826-4111-9e5c-51434f6d33f1',
 '4b6ebd02-0ce1-

In [15]:
def render_vector_store_result(result):
    for res, score in result:
        print(f'{res.page_content}\nSimilarity Search: {score}')

In [16]:
result = vector_store_flat.similarity_search_with_score('What is LLM?', k=3)

render_vector_store_result(result)

been trained on.
Medicine: The application of LLMs in the field of medicine is
reshaping healthcare delivery and research. For example, LLMs
are increasingly used in clinical decision support systems to
provide physicians with evidence-based treatment recommen-
dations [426, 427, 428]. By analyzing patient data and medical
literature, they can help identify potential diagnoses, suggest
appropriate tests, and recommend optimal treatment strategies.
Similarity Search: 0.5767259001731873
on a broad range of LLM-related concepts. Our self-contained comprehensive overview of LLMs discusses relevant background
concepts along with covering the advanced topics at the frontier of research in LLMs. This review article is intended to not only
provide a systematic survey but also a quick comprehensive reference for the researchers and practitioners to draw insights from
extensive informative summaries of the existing works to advance the LLM research.
Keywords:
Similarity Search: 0.559973597526550

In [18]:
vector_store_hnsw = Milvus(
    embedding_function=embeddings,
    connection_args={'uri': URI, 'token': os.getenv('ZILLIZ_TOKEN'), 'db_name': 'milvus_hnsw_demo'},
    index_params={'index_type': 'HNSW', 'metric_type': 'COSINE'},
    consistency_level='Strong',
    drop_old=True
)

2025-06-05 22:23:20,716 [DEBUG][_create_connection]: Created new connection using: 191ce72264ed478daeeec960fe8561f3 (async_milvus_client.py:599)


In [19]:
vector_store_hnsw.add_documents(documents=split_documents, ids=uuids)

['8c9d3420-1849-4cc6-a091-5e53d900b22d',
 '0a6dd2e4-d78b-4108-bf6c-b690cb6b455d',
 'b35dbf97-a198-4c8e-ba9b-62ea410f36d7',
 '381dee28-b8cd-4b0d-b6b0-242592689356',
 '92b74740-0d6f-4ffc-952a-7967614bc8f4',
 '8c16a930-14e1-4526-9626-d89ecb42453d',
 'dcb3cb11-7f3f-4ea2-a63e-b494566ca5b9',
 '2ca51356-ec53-4c6e-9641-c727777fe95d',
 '8b2bb7f1-cd9c-4c69-a738-3f1dcc98a61e',
 '62434b38-83d5-4ca2-a565-90e19768eecb',
 'e095daec-3764-4fbe-a421-305858ae10fc',
 '9a1a4f72-738d-433e-a7b0-a3260c0376b7',
 'b4b7a9ea-9466-4f26-bbbc-1d746c0cdf8a',
 '5a313f24-3058-4999-a011-0dacc14e9db4',
 '8dc72e18-118d-44b6-9d8a-fecf6ce67c67',
 'e90d8907-cc4c-43ea-84ac-e88645afb6b8',
 '22c400f8-3ab0-4050-bb8c-d531f08bf762',
 'b4efef44-103c-4784-9abe-734c7f5defc9',
 '4d0ec383-cf88-480b-b91d-d0cb140d36ee',
 '0206eb44-6493-49c1-8da1-d5dc070d61e5',
 'e6154186-f129-4210-bea2-d6450a6f0fa6',
 '944117d1-d14b-44b8-871a-4e77e97883eb',
 '0677598e-9109-4561-8ad7-2ff83fc85b76',
 '213fa489-1826-4111-9e5c-51434f6d33f1',
 '4b6ebd02-0ce1-

In [20]:
result = vector_store_hnsw.similarity_search_with_score('What is LLM?', k=3)

render_vector_store_result(result)

been trained on.
Medicine: The application of LLMs in the field of medicine is
reshaping healthcare delivery and research. For example, LLMs
are increasingly used in clinical decision support systems to
provide physicians with evidence-based treatment recommen-
dations [426, 427, 428]. By analyzing patient data and medical
literature, they can help identify potential diagnoses, suggest
appropriate tests, and recommend optimal treatment strategies.
Similarity Search: 0.5755277872085571
on a broad range of LLM-related concepts. Our self-contained comprehensive overview of LLMs discusses relevant background
concepts along with covering the advanced topics at the frontier of research in LLMs. This review article is intended to not only
provide a systematic survey but also a quick comprehensive reference for the researchers and practitioners to draw insights from
extensive informative summaries of the existing works to advance the LLM research.
Keywords:
Similarity Search: 0.559973597526550

In [21]:
vector_store_ivf = Milvus(
    embedding_function=embeddings,
    connection_args={'uri': URI, 'token': os.getenv('ZILLIZ_TOKEN'), 'db_name': 'milvus_ivf_demo'},
    index_params={'index_type': 'IVF_FLAT', 'metric_type': 'COSINE'},
    consistency_level='Strong',
    drop_old=True
)

2025-06-05 22:24:14,523 [DEBUG][_create_connection]: Created new connection using: 8fa65860ccf64c66ac97c4d46246276d (async_milvus_client.py:599)


In [22]:
vector_store_ivf.add_documents(documents=split_documents, ids=uuids)

['8c9d3420-1849-4cc6-a091-5e53d900b22d',
 '0a6dd2e4-d78b-4108-bf6c-b690cb6b455d',
 'b35dbf97-a198-4c8e-ba9b-62ea410f36d7',
 '381dee28-b8cd-4b0d-b6b0-242592689356',
 '92b74740-0d6f-4ffc-952a-7967614bc8f4',
 '8c16a930-14e1-4526-9626-d89ecb42453d',
 'dcb3cb11-7f3f-4ea2-a63e-b494566ca5b9',
 '2ca51356-ec53-4c6e-9641-c727777fe95d',
 '8b2bb7f1-cd9c-4c69-a738-3f1dcc98a61e',
 '62434b38-83d5-4ca2-a565-90e19768eecb',
 'e095daec-3764-4fbe-a421-305858ae10fc',
 '9a1a4f72-738d-433e-a7b0-a3260c0376b7',
 'b4b7a9ea-9466-4f26-bbbc-1d746c0cdf8a',
 '5a313f24-3058-4999-a011-0dacc14e9db4',
 '8dc72e18-118d-44b6-9d8a-fecf6ce67c67',
 'e90d8907-cc4c-43ea-84ac-e88645afb6b8',
 '22c400f8-3ab0-4050-bb8c-d531f08bf762',
 'b4efef44-103c-4784-9abe-734c7f5defc9',
 '4d0ec383-cf88-480b-b91d-d0cb140d36ee',
 '0206eb44-6493-49c1-8da1-d5dc070d61e5',
 'e6154186-f129-4210-bea2-d6450a6f0fa6',
 '944117d1-d14b-44b8-871a-4e77e97883eb',
 '0677598e-9109-4561-8ad7-2ff83fc85b76',
 '213fa489-1826-4111-9e5c-51434f6d33f1',
 '4b6ebd02-0ce1-

In [23]:
result = vector_store_ivf.similarity_search_with_score('What is LLM?', k=3)

render_vector_store_result(result)

been trained on.
Medicine: The application of LLMs in the field of medicine is
reshaping healthcare delivery and research. For example, LLMs
are increasingly used in clinical decision support systems to
provide physicians with evidence-based treatment recommen-
dations [426, 427, 428]. By analyzing patient data and medical
literature, they can help identify potential diagnoses, suggest
appropriate tests, and recommend optimal treatment strategies.
Similarity Search: 0.5776178240776062
on a broad range of LLM-related concepts. Our self-contained comprehensive overview of LLMs discusses relevant background
concepts along with covering the advanced topics at the frontier of research in LLMs. This review article is intended to not only
provide a systematic survey but also a quick comprehensive reference for the researchers and practitioners to draw insights from
extensive informative summaries of the existing works to advance the LLM research.
Keywords:
Similarity Search: 0.559973597526550

In [25]:
from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings

vectorstore = Milvus.from_documents(
    documents=split_documents,
    embedding=embeddings,
    builtin_function=BM25BuiltInFunction(),
    vector_field=['dense', 'sparse'],
    connection_args={'uri': URI, 'token': os.getenv('ZILLIZ_TOKEN')},
    consistency_level='Strong',
    drop_old=True,
)

2025-06-05 22:25:50,545 [DEBUG][_create_connection]: Created new connection using: 4bc07396a3c74570be11d2a2e38ccba4 (async_milvus_client.py:599)


In [30]:
result = vectorstore.similarity_search_with_score(
    'Explain attention in transformers', k=3, ranker_type='weighted', ranker_params={'weights': [0.6, 0.4]}
)

render_vector_store_result(result)

favors using recent tokens for attention.
RoPE [66]: It rotates query and key representations at an an-
gle proportional to the token absolute position in the input
sequence, resulting in a relative positional encoding scheme
which decays with the distance between the tokens.
2.3. Attention in LLMs
Attention assigns weights to input tokens based on impor-
tance so that the model gives more emphasis to relevant tokens.
Attention in transformers [64] calculates query, key, and value
Similarity Search: 0.706367015838623
pendency (model-specific vs. model-agnostic). Ante-hoc methods integrate interpretability during training,
while post-hoc approaches explain pre-trained models through techniques like SHAP (Shapley Additive Ex-
planations), LIME (Local Interpretable Model-Agnostic Explanations), and attention visualization (Saranya
2
Similarity Search: 0.3731301724910736
//doi.org/10.1007/s10462-022-10128-1 .
J. Sakarvadia et al. Attention lens: Exploring attention maps in transformers. ar

In [31]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template='You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know, don\'t make up any answer. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:',
    input_variables=['context', 'question']
)

In [32]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [33]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq


model = ChatGroq(model='gemma2-9b-it')

In [34]:
rag_chain = (
    {'context': vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [35]:
query = 'What is an LLM and how does LangChain help with that?'

result = rag_chain.invoke(query)

print(result)

LLM stands for Large Language Model, a type of artificial intelligence that excels at understanding and generating human-like text.  LangChain helps with LLMs by providing tools and frameworks to build and deploy applications powered by these models.  It simplifies tasks like connecting LLMs to external data sources and creating complex conversational flows.  



In [36]:
from docx import Document

doc = Document()

doc.add_heading('LangChain RAG', level=1)

doc.add_paragraph(query)

doc.add_paragraph(result)

doc.save('output.docx')